In [ ]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.3 MB/s eta 0:00:00


In [ ]:
import requests
from collections import defaultdict
import json
import re
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
news_dataset_path = '/content/drive/MyDrive/AI4E/Project_ML NLP/data/news_final.json'
with open(news_dataset_path, 'r') as fIn:
    data = json.load(fIn)
data[0], len(data)

({'url': 'https://vietnamnet.vn/kon-tum-bao-ton-phuc-dung-nep-nha-rong-truyen-thong-2257205.html',
  'title': 'Kon Tum bảo tồn, phục dựng nếp nhà rông truyền thống',
  'description': 'Cuối năm 2023, dân làng Đăk Đe, xã Rờ Kơi, huyện Sa Thầy (Kon Tum) đã cùng nhau họp bàn, phân công nhiệm vụ cho từng hộ gia đình, từng cá nhân tìm các nguyên, vật liệu để dựng nhà rông truyền thống.',
  'content': 'Với cộng đồng các dân tộc thiểu số ở Tây Nguyên nói chung và Kon Tum nói riêng, nhà rông là “trái tim” của làng, nơi hội tụ tinh hoa văn hóa truyền thống được bà con ra sức gìn giữ, phát huy.\nTuy nhiên, hiện nay, vì nhiều lý do như khó khăn trong công tác quản lý, thiếu các vật liệu xây dựng,… mà nhà rông truyền thống gần như không còn. Thay vào đó là các nhà rông văn hoá được xây dựng bằng các vật liệu hiện đại. Điều này không chỉ dẫn đến nguy cơ mai một về văn hoá, mà còn khiến bà con không còn quá mặn mà với việc đến nhà rông sinh hoạt.\nĐược hỗ trợ gần 100 triệu đồng từ Dự án 6 Chương trìn

In [ ]:
# Stopwords tiếng việt
stop_words = set([
    "bị", "bởi", "cả", "các", "cái", "cần", "càng", "chỉ", "chiếc", "cho", "chứ", "chưa", "chuyện",
    "có", "có_thể", "cứ", "của", "cùng", "cũng", "đã", "đang", "đây", "để", "đến_nỗi", "đều", "điều",
    "do", "đó", "được", "dưới", "gì", "khi", "không", "là", "lại", "lên", "lúc", "mà", "mỗi", "này",
    "nên", "nếu", "ngay", "nhiều", "như", "nhưng", "những", "nơi", "nữa", "phải", "qua", "ra", "rằng",
    "rằng", "rất", "rồi", "sau", "sẽ", "so", "sự", "tại", "theo", "thì", "trên", "trước", "từ", "từng",
    "và", "vẫn", "vào", "vậy", "vì", "việc", "với", "vừa"
])
def preprocess_text(text):
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    # Chia thành các token
    tokens = text.split()
    # Loại bỏ stopwords
    tokens = [token for token in tokens if token.lower() not in stop_words]
    # Văn bản đã được xử lí
    text = ' '.join(tokens)
    return text

In [ ]:
def process_entity(entity):
    count = 0
    for char in entity:
        if char.isalnum():
            break
        else:
            count+=1
    return entity[count:].upper()

In [ ]:
process_entity(',[ UKRAINE')

'UKRAINE'

In [ ]:
from underthesea import ner
news_NER = defaultdict(set)
for doc in tqdm(data, desc="Processing: "):
        text = doc['title'] + ' ' + doc['content']
        processed_text = preprocess_text(text)
        ner_results = ner(processed_text)
        current_entity = None
        current_label = None

        for entity in ner_results:
            if entity[3].startswith('B-'):
                if current_entity is not None:
                    news_NER[process_entity(current_entity)].add(doc['url'])
                current_entity = entity[0]
                current_label = entity[3]
            elif entity[3].startswith('I-') and current_entity is not None:
                current_entity += ' ' + entity[0]
            else:
                if current_entity is not None:
                    news_NER[process_entity(current_entity)].add(doc['url'])
                    current_entity = None
                    current_label = None
        if current_entity is not None:
            news_NER[process_entity(current_entity)].add(doc['url'])

In [ ]:
news_NER_items = news_NER.items()
news_NER_sorted = sorted(news_NER_items, key=lambda item: len(item[1]), reverse=True)
news_NER = dict(news_NER_sorted)

In [ ]:
ner_path = '/content/drive/MyDrive/AI4E/Project_ML NLP/NER/ner.json'
with open(ner_path, 'w') as fOut:
    json.dump({k: list(v) for k, v in news_NER.items()}, fOut, indent=4, ensure_ascii=False)